In [4]:
import socket
import tkinter as tk
from tkinter import messagebox
from threading import Thread
import numpy as np
import cv2
import json
from io import BytesIO
from PIL import Image
import time

# your server code
def run_server(ip, port):
    # convert the port from string to int
    port = int(port)

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((ip, port))
    server_socket.listen(1)
    net = cv2.dnn.readNet('yolov3.cfg', "/home/davorin/Documents/OP/yolov3.weights")


    def recv_all(sock, n):
    # Helper function to receive n bytes or return None if EOF is hit
        data = bytearray()
        while len(data) < n:
            packet = sock.recv(n - len(data))
            if not packet:
                return None
            data.extend(packet)
        return data
    
    def calculate_ap(precision, recall):
    # Sort precision and recall in descending order of confidence scores
        sorted_indices = np.argsort(-precision)
        sorted_precision = precision[sorted_indices]
        sorted_recall = recall[sorted_indices]

    # Compute interpolated precision values
        for i in range(len(sorted_precision) - 2, -1, -1):
            sorted_precision[i] = max(sorted_precision[i], sorted_precision[i + 1])

    # Compute area under the precision-recall curve
        ap = np.sum((sorted_recall[1:] - sorted_recall[:-1]) * sorted_precision[:-1])

        return ap
    
    def calculate_iou(boxA, boxB):
    # Calculate coordinates of the intersection rectangle
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

    # Compute the intersection area
        intersection_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # Compute the union area
        boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
        union_area = boxA_area + boxB_area - intersection_area

    # Calculate IoU
        iou = intersection_area / union_area

        return iou
    
    while True:
        print("Waiting for a connection...")
        client_socket, addr = server_socket.accept()
        print(f"Accepted connection from {addr}")

    # receive the image data
        length_prefix = recv_all(client_socket, 4)  # assuming that the length is sent as 4 bytes (a 32-bit int)
        if length_prefix is None:
            print('Length prefix not received')
        else:
            length = int.from_bytes(length_prefix, 'little')  # convert bytes to int
            data = recv_all(client_socket, length)  # receive the rest of the data
            if data is None:
                print('Image not received')
            else:
                print("Loading classes")
                with open("coco.names", "r") as f:
                    classes = [line.strip() for line in f.readlines()]
                print("Classes loaded")

            # convert the bytes to an image
                image_stream = BytesIO(data)
                image = Image.open(image_stream)
                image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

            # use the YOLO model to perform object detection
                print("Image is being processed")
                height, width = image.shape[:2]
                blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
                net.setInput(blob)
                start_time = time.time()
                outs = net.forward(net.getUnconnectedOutLayersNames())
                end_time = time.time()
                execution_time = end_time - start_time
                print("Execution Time: {:.2f} seconds".format(execution_time))

                print("Inference running")
           
            # process the results
                class_ids = []
                confidences = []
                boxes = []
                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.5:
                            center_x = int(detection[0] * width)
                            center_y = int(detection[1] * height)
                            w = int(detection[2] * width)
                            h = int(detection[3] * height)
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)
                print("Coordinates are here:")

            # apply non-maxima suppression to get the most relevant detections
                indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            # create a list to store all the detection results
                results = []

                if len(indices) > 0:
                    for i in indices.flatten():  # flatten the list to handle single/multiple indices
                        box = boxes[i]
                        x, y, w, h = box[0], box[1], box[2], box[3]
                        label = str(classes[class_ids[i]])
                        result = {'label': label, 'x': x, 'y': y, 'w': w, 'h': h}
                        results.append(result)

                num_objects = len(results)
                print("Number of objects:")
                print(num_objects)
            # Calculate Average Precision (AP)
                if len(indices) > 0:
                    precision = np.ones(len(indices))
                    recall = np.arange(1, len(indices) + 1) / len(indices)
                    ap = calculate_ap(precision, recall)
                    print("Average Precision (AP):", ap)
            
            # Add execution time and AP to the results dictionary
                    results.append({'execution_time': execution_time, 'ap': ap, 'num_objects': num_objects})

                print(results)
        # send the detection results back to the client
            client_socket.sendall(json.dumps(results).encode())

        client_socket.close()

    

    # add the rest of your server code here...

class VisionSphereApp(tk.Tk):
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Vision Sphere Application")
        self.root.geometry('600x400')  # Set the window size to 600x400

        self.ip_entry = tk.Entry(self.root, width=15)
        self.ip_entry.pack(padx=20, pady=20)  # Added padding for a cleaner layout

        self.port_entry = tk.Entry(self.root, width=5)
        self.port_entry.pack(padx=20, pady=20)

        self.start_button = tk.Button(self.root, text="Start server", command=self.start_server)
        self.start_button.pack(padx=20, pady=20)

    def start_server(self):
        ip = self.ip_entry.get()
        port = self.port_entry.get()

        if not ip or not port:
            messagebox.showinfo("Missing info", "IP and Port are required to start the server.")
        else:
            thread = Thread(target=run_server, args=(ip, port))
            thread.start()
            messagebox.showinfo("Server status", "Server started.")
    
    def run(self):
        self.root.mainloop()



In [6]:

app = VisionSphereApp()
app.run()

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/davorin/anaconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/davorin/anaconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_5951/1224416876.py", line 18, in run_server
OSError: [Errno 98] Address already in use


Waiting for a connection...
Accepted connection from ('192.168.0.215', 58334)
Loading classes
Classes loaded
Image is being processed
Execution Time: 0.33 seconds
Inference running
Coordinates are here:
Number of objects:
9
Average Precision (AP): 0.8888888888888888
[{'label': 'car', 'x': 284, 'y': 207, 'w': 181, 'h': 146}, {'label': 'car', 'x': 557, 'y': 201, 'w': 259, 'h': 155}, {'label': 'car', 'x': 464, 'y': 215, 'w': 124, 'h': 123}, {'label': 'person', 'x': 155, 'y': 150, 'w': 135, 'h': 221}, {'label': 'car', 'x': 757, 'y': 204, 'w': 110, 'h': 139}, {'label': 'car', 'x': 1, 'y': 158, 'w': 92, 'h': 195}, {'label': 'car', 'x': 421, 'y': 224, 'w': 53, 'h': 77}, {'label': 'bicycle', 'x': 196, 'y': 298, 'w': 96, 'h': 146}, {'label': 'truck', 'x': 104, 'y': 166, 'w': 100, 'h': 143}, {'execution_time': 0.32871508598327637, 'ap': 0.8888888888888888, 'num_objects': 9}]
Waiting for a connection...
